In [234]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

In [235]:
def removeUnamed(df):
    columnList = df.columns.tolist()
    newColumnList = columnList.copy()
    i = 0
    while i<len(newColumnList):
        if "Unnamed" in newColumnList[i]:
            stats = newColumnList[i-1].split(" / ")
            newColumnList[i-1] = stats[0]
            newColumnList[i] = " ".join(stats)
            newColumnList[i+1] = " ".join(stats) + " percentage"
            
        if "Losses" in newColumnList[i] or "Recoveries" in newColumnList[i]:
            stats = newColumnList[i].split(" / ")
            newColumnList[i] = stats[0]
            newColumnList[i+1] = stats[0] + " "+stats[1]
            newColumnList[i+2] = stats[0] + " "+stats[2]
            newColumnList[i+3] = stats[0] + " "+stats[3]
            i+=3
        i+=1
    columnsRenamed = dict(zip(columnList,newColumnList))
    df = df.rename(columns=columnsRenamed)
    return df
    

In [236]:
def removePercents(df):
    columns = df.columns.tolist()
    for i in range(len(columns)):
        if "percent" in columns[i]:
            df.drop(columns[i],axis=1,inplace=True)
    return df

In [237]:
# files = [file for file in os.listdir()[:-2] if "csv" in file]

In [238]:
folders = [file for file in os.listdir() if "." not in file]
minutes = ["1-15","16-30","31-45","46-60","61-75","76-90","WholeGameData"]
df_dict = {"1-15":[],"16-30":[],"31-45":[],"46-60":[],"61-75":[],"76-90":[],"WholeGameData":[]}
for folder in folders:
    files = os.listdir(folder)
    files = [file for file in files if ("csv" in file) and (("AllStats" in file) or ("WholeGameData" in file))]
    for file in files:
        df = pd.read_csv(folder+"/"+file)
        df = removeUnamed(df)
        df = removePercents(df)
        if "WholeGameData" in file:
            df_dict["WholeGameData"].append(df)
        else:
            for val in minutes:
                if val in file:
                    df["Minute"] = int(val.split("-")[0])
                    minute = val
            df_dict[minute].append(df)



In [239]:
for val in minutes:
    merged = df_dict[val][0]
    for df in df_dict[val][1:]:
        merged = pd.merge(merged,df,how="outer")
    df_dict[val] = merged

In [240]:
merged = list(df_dict.values())[0]
for df in list(df_dict.values())[1:-1]:
    merged = pd.merge(merged,df,how="outer")

In [241]:
wholeGameData = df_dict["WholeGameData"]
wholeGameData = wholeGameData[wholeGameData.get("Date").apply(lambda date:"2023" in date)]
leagueGames = wholeGameData[wholeGameData.get("Competition") == "United States. NCAA D1 Big West"].drop("Competition",axis =1)
seasonSums = wholeGameData.drop(["Date","Scheme","Match","Competition"],axis = 1).groupby("Team").sum()

In [242]:
wholeGameData

,Date,Match,Competition,Duration,Team,Scheme,Goals,xG,Shots,Shots on target,...,Smart passes accurate,Throw ins,Throw ins accurate,Goal kicks,Match tempo,Average passes per possession,Long pass %,Average shot distance,Average pass length,PPDA
68,2023-08-20,UC Santa Barbara Gauchos - Westmont Warriors 5:1,United States. NCAA D1 Non-conference matches,99,UC Santa Barbara Gauchos,4-2-3-1 (100.0%),5,4.22,21,12,...,7,19,18,4,16.11,3.81,9.74,17.05,21.11,6.29
69,2023-08-20,UC Santa Barbara Gauchos - Westmont Warriors 5:1,United States. NCAA D1 Non-conference matches,99,Westmont Warriors,4-1-4-1 (70.4%),1,0.30,7,2,...,1,19,13,10,14.36,3.31,9.55,19.45,19.16,16.24
70,2023-08-24,UC Irvine Anteaters - UNLV Rebels 1:0,United States. NCAA D1 Non-conference matches,108,UC Irvine Anteaters,4-2-3-1 (100.0%),1,1.66,10,2,...,0,35,35,8,17.75,4.34,8.27,16.53,19.31,9.07
71,2023-08-24,UC Irvine Anteaters - UNLV Rebels 1:0,United States. NCAA D1 Non-conference matches,108,UNLV Rebels,4-2-3-1 (78.06%),0,0.26,4,1,...,0,24,20,11,15.47,3.21,12.76,17.01,20.11,11.15
72,2023-08-25,CSU Bakersfield Roadrunners - San Jose State S...,United States. NCAA D1 Non-conference matches,99,CSU Bakersfield Roadrunners,4-1-4-1 (100.0%),1,1.16,16,6,...,0,20,11,9,14.29,3.31,9.89,20.88,19.45,10.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,2023-11-05,UC Irvine Anteaters - UC Santa Barbara Gauchos...,United States. NCAA D1 Big West,105,UC Santa Barbara Gauchos,3-4-1-2 (34.52%),1,0.45,8,3,...,1,40,34,8,17.13,5.29,8.32,22.77,18.26,7.09
320,2023-11-12,UC Irvine Anteaters - UC Davis Aggies (P) 0:0,United States. NCAA D1 Big West,122,UC Davis Aggies,4-2-3-1 (100.0%),0,1.03,17,2,...,0,41,37,9,13.70,2.79,17.45,21.12,20.31,8.98
321,2023-11-12,UC Irvine Anteaters - UC Davis Aggies (P) 0:0,United States. NCAA D1 Big West,122,UC Irvine Anteaters,4-2-3-1 (90.5%),0,1.11,13,4,...,1,29,28,16,16.24,3.37,10.60,19.14,20.33,9.38
322,2023-11-17,LMU Lions - UC Irvine Anteaters 4:2,United States. NCAA D1 College Cup,119,LMU Lions,4-1-4-1 (74.58%),4,3.77,13,7,...,1,15,13,11,14.92,2.50,17.33,14.95,22.02,11.20


In [243]:
for df in [merged, wholeGameData,seasonSums]:
    
    #inserting percentages
    df.insert(19, "Penalty conversion percentage",df.get("Penalties converted")/df.get("Penalties") )
    df.insert(22, "Crossing accuracy",df.get("Crosses accurate")/df.get("Crosses") )
    df.insert(9, "Shots on target accuracy",df.get("Shots on target")/df.get("Shots") )
    df.insert(78, "Throw in percentage",df.get("Throw ins accurate")/df.get("Throw ins") )
    df.insert(76, "Smart passes accuracy",df.get("Smart passes accurate")/df.get("Smart passes") )
    df.insert(74, "Progressive passes accuracy",df.get("Progressive passes accurate")/df.get("Progressive passes") )
    df.insert(72, "Passes to final third accuracy",df.get("Passes to final third accurate")/df.get("Passes to final third") )
    df.insert(70, "Long passes accuracy",df.get("Long passes accurate")/df.get("Long passes") )
    df.insert(68, "Lateral passes accuracy",df.get("Lateral passes accurate")/df.get("Lateral passes") )
    df.insert(66, "Back passes accuracy",df.get("Back passes accurate")/df.get("Back passes") )
    df.insert(64, "Forward passes accuracy",df.get("Forward passes accurate")/df.get("Forward passes") )
    df.insert(56, "Duels accuracy",df.get("Duels won")/df.get("Duels") )
    df.insert(45, "Passes accuracy",df.get("Passes accurate")/df.get("Passes") )
    df.insert(37, "Sliding tackle percentage",df.get("Sliding tackles successful")/df.get("Sliding tackles") )
    df.insert(35, "Aerial duels accuracy",df.get("Aerial duels won")/df.get("Aerial duels") )
    df.insert(33, "Defensive duels accuracy",df.get("Defensive duels won")/df.get("Defensive duels") )
    df.insert(31, "Shots against on target accuracy",df.get("Shots against on target")/df.get("Shots against") )
    df.insert(26, "Offensive duels accuracy",df.get("Offensive duels won")/df.get("Offensive duels") )
    df.insert(16, "Corners with shots percentage",df.get("Corners with shots")/df.get("Corners") )
    df.insert(16, "Counterattacks with shots percentage",df.get("Counterattacks with shots")/df.get("Counterattacks") )
    df.insert(16, "Shots from outside penalty area on target accuracy",df.get("Shots from outside penalty area on target")/df.get("Shots from outside penalty area") )
    df.insert(16, "Positional attacks with shots percentage",df.get("Positional attacks with shots")/df.get("Positional attacks") )
    df.insert(16, "Shots from outside penalty area on target percentage",df.get("Shots from outside penalty area on target")/df.get("Shots on target") )
    df.insert(64, "Forward passes percentage",df.get("Forward passes accurate")/df.get("Passes accurate") )
    df.insert(64, "Back passes percentage",df.get("Back passes accurate")/df.get("Passes accurate") )
    df.insert(64, "Lateral passes percentage",df.get("Lateral passes accurate")/df.get("Passes accurate") )
    df.insert(64, "Smart passes percentage",df.get("Smart passes accurate")/df.get("Passes accurate") )
    df.insert(64, "Passes to final third percentage",df.get("Passes to final third accurate")/df.get("Passes accurate") )
    df.insert(64, "Progressive passes percentage",df.get("Progressive passes accurate")/df.get("Passes accurate") )
    df.insert(64, "Long passes percentage",df.get("Long passes accurate")/df.get("Passes accurate") )
    #inserting custom stats
    df.insert(7, "Difference in Goals and xG",df.get("Goals")-df.get("xG") )
    df.insert(8, "Goal differential", df.get("Goals")-df.get("Conceded goals"))

C:\Users\zcoch\AppData\Local\Temp\ipykernel_10504\4266058181.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(26, "Offensive duels accuracy",df.get("Offensive duels won")/df.get("Offensive duels") )
C:\Users\zcoch\AppData\Local\Temp\ipykernel_10504\4266058181.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(16, "Corners with shots percentage",df.get("Corners with shots")/df.get("Corners") )
C:\Users\zcoch\AppData\Local\Temp\ipykernel_10504\4266058181.py:23: PerformanceWarning: DataFrame is highly frag

In [244]:
leagueGames.insert(8, "Goal differential", leagueGames.get("Goals")-leagueGames.get("Conceded goals"))

In [245]:
percantagesColumns =[column for column in seasonSums.columns.tolist() if ("accuracy" in column) or ("percentage" in column)] + ["Difference in Goals and xG"]
seasonPercentages = seasonSums.get(percantagesColumns).reset_index()
seasonAverages = leagueGames.drop(["Date","Scheme","Match"],axis = 1).groupby("Team").mean().reset_index()
seasonStats = pd.merge(seasonAverages,seasonPercentages,on = "Team")

In [246]:
seasonStats.columns.tolist()

['Team',
 'Duration',
 'Goals',
 'xG',
 'Shots',
 'Goal differential',
 'Shots on target',
 'Passes',
 'Passes accurate',
 'Possession, %',
 'Losses',
 'Losses Low',
 'Losses Medium',
 'Losses High',
 'Recoveries',
 'Recoveries Low',
 'Recoveries Medium',
 'Recoveries High',
 'Duels',
 'Duels won',
 'Shots from outside penalty area',
 'Shots from outside penalty area on target',
 'Positional attacks',
 'Positional attacks with shots',
 'Counterattacks',
 'Counterattacks with shots',
 'Corners',
 'Corners with shots',
 'Free kicks',
 'Free kicks with shots',
 'Penalties',
 'Penalties converted',
 'Crosses',
 'Crosses accurate',
 'Deep completed crosses',
 'Deep completed passes',
 'Penalty area entries (runs',
 'Penalty area entries (runs crosses)',
 'Touches in penalty area',
 'Offensive duels',
 'Offensive duels won',
 'Offsides',
 'Conceded goals',
 'Shots against',
 'Shots against on target',
 'Defensive duels',
 'Defensive duels won',
 'Aerial duels',
 'Aerial duels won',
 'Sliding

In [247]:
bigWestTeams = wholeGameData[wholeGameData.get("Competition")=="United States. NCAA D1 Big West"].get("Team").unique()
points = [10,19,12,13,11,7,16,12,10,14]
pointsDict = dict(zip(bigWestTeams,points))
def addPoints(team):
    if team in bigWestTeams:
        return pointsDict[team]
    else:
        return 0
seasonStats.insert(1,"Points",seasonStats.get("Team").apply(addPoints))
seasonStats.set_index("Team")
def addGamePoints(gd):
    if gd>0:
        return 3
    elif gd<0:
        return 0
    else:
        return 1
wholeGameData.insert(6,"Points",wholeGameData.get("Goal differential").apply(addGamePoints))

C:\Users\zcoch\AppData\Local\Temp\ipykernel_10504\1861761239.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  wholeGameData.insert(6,"Points",wholeGameData.get("Goal differential").apply(addGamePoints))


In [248]:
columns = merged.columns.tolist()
first = columns[:6]
second = sorted(columns[6:])
merged = merged.get(first+second)
columns = wholeGameData.columns.tolist()
first = columns[:7]
second = sorted(columns[7:])
wholeGameData = wholeGameData.get(first+second)
columns = seasonStats.columns.tolist()
first = columns[:3]
second = sorted(columns[3:])
seasonStats = seasonStats.get(first+second)
wholeGameData

,Date,Match,Competition,Duration,Team,Scheme,Points,Aerial duels,Aerial duels accuracy,Aerial duels won,...,Smart passes,Smart passes accuracy,Smart passes accurate,Smart passes percentage,Throw in percentage,Throw ins,Throw ins accurate,Touches in penalty area,Yellow cards,xG
68,2023-08-20,UC Santa Barbara Gauchos - Westmont Warriors 5:1,United States. NCAA D1 Non-conference matches,99,UC Santa Barbara Gauchos,4-2-3-1 (100.0%),3,26,0.653846,17,...,11,0.636364,7,0.019663,0.947368,19,18,33,2,4.22
69,2023-08-20,UC Santa Barbara Gauchos - Westmont Warriors 5:1,United States. NCAA D1 Non-conference matches,99,Westmont Warriors,4-1-4-1 (70.4%),0,26,0.192308,5,...,3,0.333333,1,0.003195,0.684211,19,13,14,0,0.30
70,2023-08-24,UC Irvine Anteaters - UNLV Rebels 1:0,United States. NCAA D1 Non-conference matches,108,UC Irvine Anteaters,4-2-3-1 (100.0%),3,34,0.323529,11,...,0,NaN,0,0.000000,1.000000,35,35,19,3,1.66
71,2023-08-24,UC Irvine Anteaters - UNLV Rebels 1:0,United States. NCAA D1 Non-conference matches,108,UNLV Rebels,4-2-3-1 (78.06%),0,34,0.588235,20,...,2,0.000000,0,0.000000,0.833333,24,20,7,1,0.26
72,2023-08-25,CSU Bakersfield Roadrunners - San Jose State S...,United States. NCAA D1 Non-conference matches,99,CSU Bakersfield Roadrunners,4-1-4-1 (100.0%),1,25,0.360000,9,...,0,NaN,0,0.000000,0.550000,20,11,15,4,1.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,2023-11-05,UC Irvine Anteaters - UC Santa Barbara Gauchos...,United States. NCAA D1 Big West,105,UC Santa Barbara Gauchos,3-4-1-2 (34.52%),0,41,0.414634,17,...,2,0.500000,1,0.002045,0.850000,40,34,12,2,0.45
320,2023-11-12,UC Irvine Anteaters - UC Davis Aggies (P) 0:0,United States. NCAA D1 Big West,122,UC Davis Aggies,4-2-3-1 (100.0%),1,53,0.415094,22,...,0,NaN,0,0.000000,0.902439,41,37,27,3,1.03
321,2023-11-12,UC Irvine Anteaters - UC Davis Aggies (P) 0:0,United States. NCAA D1 Big West,122,UC Irvine Anteaters,4-2-3-1 (90.5%),1,53,0.433962,23,...,2,0.500000,1,0.002538,0.965517,29,28,21,2,1.11
322,2023-11-17,LMU Lions - UC Irvine Anteaters 4:2,United States. NCAA D1 College Cup,119,LMU Lions,4-1-4-1 (74.58%),3,33,0.606061,20,...,2,0.500000,1,0.004274,0.866667,15,13,12,5,3.77


In [249]:
wholeGameData =wholeGameData[wholeGameData.get("Date").apply(lambda date: "2023" in date)]

In [250]:
merged.to_csv("DataByMinute.csv",index=False)
wholeGameData.to_csv("WholeGameData.csv",index = False)
seasonStats.to_csv("SeasonStats.csv",index = False)

In [251]:
season

NameError: name 'season' is not defined